In [ ]:
import os, json, pandas as pd, yaml
from pathlib import Path
from utils.graph import build_from_snapshot, Graph
from utils.episodes import build_episodes, load_rules, apply_rules, episode_to_incident

DATA_DIR = Path(os.environ.get("DATA_DIR", "data"))
OUT_DIR  = Path(os.environ.get("OUT_DIR", "incidents"))
OUT_DIR.mkdir(parents=True, exist_ok=True)
RULES_PATH = "rules/rules.yaml"


In [ ]:
# Expect a parquet or CSV that already merges audit/app/infra with a common schema.
# Required columns: ts, source, namespace, pod, node, level, code, verb, msg, (optionally) rollout_in_window, container_restart
df = pd.read_parquet(DATA_DIR / "unified_logs/latest.parquet")
df.head()

In [ ]:
snapshot = json.loads((DATA_DIR / "topology_snapshot.json").read_text())
graph: Graph = build_from_snapshot(snapshot)
print(f"Graph nodes={len(graph.meta)} edges={sum(len(v) for v in graph.adj.values())}")

In [ ]:
episodes = build_episodes(df, window="10min", keys=["namespace","pod","node"])
len(episodes), episodes[0]

In [ ]:
rules = load_rules(RULES_PATH)
written = 0
for ep in episodes:
    cands = apply_rules(ep, rules, graph)
    incident = episode_to_incident(ep, cands)
    out = OUT_DIR / f"{ep.episode_id}.json"
    out.write_text(json.dumps(incident, indent=2))
    written += 1
written

In [ ]:
tbl = []
for f in OUT_DIR.glob("*.json"):
    inc = json.loads(f.read_text())
    top = inc["candidates"][0] if inc["candidates"] else None
    tbl.append({
        "episode_id": inc["episode_id"],
        "start": inc["start"],
        "end": inc["end"],
        "count": inc["features"].get("count", 0),
        "error_ratio": inc["features"].get("error_ratio", 0.0),
        "top_component": top["component"] if top else None,
        "top_reason": top["reason"] if top else None,
        "top_score": top["score"] if top else 0.0,
    })
pd.DataFrame(tbl).to_parquet(OUT_DIR / "incidents_index.parquet", index=False)
pd.DataFrame(tbl).head()